In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks

print(tf.__version__)

2.12.0


In [3]:
import keras_tuner as kt

In [4]:
from eggman.grid import luminosity, log_surface_gravity, calculate_eep

In [ ]:
# TODO: define normalizer, num_labels, and rescaler

In [ ]:
def build_model(hp):
    units = hp.Int("units_per_layer", min_value=32, max_value=512, step=32)
    num_layers = hp.Int("hidden_layers", min_value=3, max_value=8, step=1)
    model = tf.keras.Sequential(
        [normalizer] +
        [layers.Dense(units, "elu") for _ in range(num_layers)] +
        [layers.Dense(num_labels), rescaler]
    )
    
    
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    opt = optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999)
    loss = 'mean_squared_error'

    model.compile(opt, loss=loss)
    return model

In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=100,
    factor=3,
    directory='logs',
    project_name='stellar-emulator'
)

In [ ]:
stop_early = callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
epochs = 50
batch_size = 2048

In [ ]:
tuner.search(
    img_train,
    label_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    callbacks=[stop_early]
)